Импорт библиотек

In [199]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

Функция для обработки выбросов

In [200]:
def remove_outliers(df, cols, threshold=1.5):
    for col in cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

Загружаю данные

In [201]:
data = pd.read_csv('data/train.csv')

Привожу данные к виду подходящему для дальнейшей обработки

Удаляю пустые строки

In [202]:
data = data.dropna(how='all')

Переведу временные столбцы в числовые значения

In [203]:
if 'ApplicationDate' in data.columns:
    data['ApplicationDate'] = pd.to_datetime(data['ApplicationDate']).astype(int) // 10**9


Удаляем выбросы

In [204]:
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
data = remove_outliers(data, numeric_columns)

Разделяем датасет на признаки и результаты

In [205]:
X = data.drop(columns=['RiskScore'])
y = data['RiskScore']

Разделяю на категориальные признаки и числовые

In [206]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

Далее делаю пайплайн обработки датасета по столбцам

Обработка численных значений

In [207]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

Обработка категориальных переменных

In [208]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

Обработка по столбцам

In [209]:
process = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

Модель линейной регресии - обработка - линейная регрессия

In [210]:
model = Pipeline(steps=[
    ('process', process),
    ('regressor', LinearRegression())
])

Разделение на обучающую и тестовую выборку

In [211]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Обучаю модель

In [212]:
model.fit(X_train, y_train)

Pipeline(steps=[('process',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['ApplicationDate', 'Age', 'AnnualIncome', 'CreditScore', 'LoanAmount',
       'LoanDuration', 'NumberOfDependents', 'MonthlyDebtPayments',
       'CreditCardUtilizationRate', 'NumberOfOpenCreditLines',
       'NumberO...
       'InterestRate', 'MonthlyLoanPayment', 'TotalDebtToIncomeRatio',
       'LoanApproved'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  Index(['MaritalStatus', 'HomeOwnershipStatus', 'LoanPurpose',
       'EmploymentStatus', 'EducationLevel'],
      dtype='object'))])),
                ('regressor', LinearRegression())])

Делаю предсказания, вычисляю ошибки

In [213]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print(f'Train-MSE: {mse_train}')
print(f' Test-MSE: {mse_test}')

Train-MSE: 14.325997222317213
 Test-MSE: 15.720379674372854


Загружаю тестовый датасет

In [214]:
test_data = pd.read_csv('data/test.csv')

Привожу данные к стандартному виду

In [215]:
test_data = test_data.dropna(how='all')
if 'ApplicationDate' in test_data.columns:
    test_data['ApplicationDate'] = pd.to_datetime(test_data['ApplicationDate']).astype(int) // 10**9
numeric_columns = test_data.select_dtypes(include=['float64', 'int64']).columns
test_data = remove_outliers(test_data, numeric_columns)


Делаем предсказания

In [216]:
prediction = model.predict(test_data)

Сохраняем, выгружаем

In [217]:
predictions_df = pd.DataFrame({
    'ID': test_data.index,
    'RiskScore': prediction
})


predictions_df.to_csv('res.csv', index=False)